# Supplementary Information: Holmes *et al.* 2017

# 2. Full model fitting

This notebook describes fitting of the Bayesian hierarchical model of the effect of control and treatment, by gene/locus tag, to the full array dataset.

## Python imports

In [1]:
%pylab inline

import os

import numpy as np
import pandas as pd
import pickle
import pystan
import scipy
import seaborn as sns

import tools

import warnings
warnings.filterwarnings('ignore')

sns.set_context('notebook')

Populating the interactive namespace from numpy and matplotlib


/Users/lpritc/Development/GitHub/Supplementary_Info/SI_Holmes_etal_2016/venv-SI_Holmes_2016/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


## Building the model

We construct our model to use groupwise (by gene ID) values for $\gamma_{j[i]}$ and $\delta_{j[i]}$, where these coefficients are each drawn from a pooled distribution covering all *gene IDs*. This assumes that the intercept and slope for the relationship between *input* and *output* has the same distribution across all *gene IDs*; this may not be true: we expect that some genes are more responsive than others, in terms of selection. We use a Cauchy distribution (which has fat tails with infinite variance) as a prior on parameter values.

We therefore construct the following model of the experiment:

$$y_i = \alpha_{j[i]} + \beta_{j[i]} x_i + \gamma_{j[i]} t_i + \delta_{j[i]} t_i x_i + \epsilon_i$$
$$\alpha_{j[i]} \sim Cauchy(\mu_{\alpha}, \sigma_{\alpha}^2)$$
$$\beta_{j[i]} \sim Cauchy(\mu_{\beta}, \sigma_{\beta}^2)$$
$$\gamma_{j[i]} \sim Cauchy(\mu_{\gamma}, \sigma_{\gamma}^2)$$
$$\delta_{j[i]} \sim Cauchy(\mu_{\delta}, \sigma_{\delta}^2)$$
$$\sigma_{\alpha} \sim U(0, 100)$$
$$\sigma_{\beta} \sim U(0, 100)$$
$$\sigma_{\gamma} \sim U(0, 100)$$
$$\sigma_{\delta} \sim U(0, 100)$$


* $y_i$: measured intensity output on the array for probe $i$ (specific to each replicate)
* $x_i$: measured intensity input on the array for probe $i$ (specific to each replicate)
* $t_i$: 0/1 indicating whether the probe $i$ was measured in a control (0) or treatment (1) experiment
* $\alpha_{j[i]}$: the linear intercept, this is a constant 'offset' for all *output* measurements relative to *input* measurements, but it differs for each *gene ID*; this is drawn from a Normal distribution $\sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$
* $\mu_{\alpha}$: the mean offset for all *gene ID*s
* $\sigma_{\alpha}$: the standard deviation of the offset for all *gene ID*s - sampled from a Uniform distribution $U(0, 100)$
* $\beta_{j[i]}$: the linear slope, this is the relative change in measured intensity between *input* and *output* intensities *at the probe ID level* - it differs for each *gene ID*; this is drawn from a Normal distribution $\sim N(\mu_{\beta}, \sigma_{\beta}^2)$
* $\mu_{\beta}$: the mean slope for all *gene ID*s
* $\sigma_{\beta}$: the standard deviation of the slope for all *gene ID*s - sampled from a Uniform distribution $U(0, 100)$
* $\gamma_{j[i]}$: a groupwise estimate of the influence on *output* measured intensity of introducing passage (*treatment*) into the experiment (i.e. an intercept correction)
* $\mu_{\gamma}$: the mean intercept due to *treatment* for each *gene ID*
* $\sigma_{\gamma}$: the standard deviation of the offset for each *gene ID* due to the *treatment* - sampled from a Uniform distribution $U(0, 100)$
* $\delta_{j[i]}$: a groupwise estimate of the influence on *output* measured intensity of introducing passage (*treatment*) into the experiment, as a function of the *input* measured intensity (i.e. a slope correction)
* $\mu_{\delta}$: the mean slope due to *treatment* for each *gene ID*
* $\sigma_{\delta}$: the standard deviation of the slope for each *gene ID* due to *treatment* - sampled from a Uniform distribution $U(0, 100)$
* $\epsilon_i$: error in the model prediction for probe $i$

### Stan model construction and fit

We need to define `data`, `parameters` and our `model` for `Stan`.

In the `data` block, we have:

* `N`: `int`, the number of data points
* `J`: `int`, the number of unique locus tags (`J` < `N`)
* `tag`: `int[N]`, an index list of locus tags
* `t`: `vector[N]`, 0/1 control/treatment values for each probe
* `x`: `vector[N]`, the input log(intensity) values
* `y`: `vector[N]`, the output log(intensity) values

In the `parameter` block, we have:

* `a`: `real vector[J]`, offset effect on log(intensity) of passing through the experiment for each locus tag/gene
* `mu_a`: `real`, an unconstrained value to be fit that represents the mean offset for all locus tags
* `b`: `real vector[J]`, effect on log(intensity) of passing through the experiment, for each locus tag/gene
* `mu_b`: `real`, an unconstrained value to be fit that represents the mean slope for all locus tags
* `g`: `real vector[J]`, estimate of the influence of treatment on the output measured intensity (offset) for each locus tag/gene
* `mu_g`: `real`, an unconstrained value to be fit that represents the mean offset for all locus tags due to *treatment*
* `d`: `real vector[J]`, estimate of the influence of treatment on the output measured intensity (slope) for each locus tag/gene
* `mu_d`: `real`, an unconstrained value to be fit that represents the mean slope for all locus tags due to *treatment*
* `sigma`: `real<lower=0>`, the error in the prediction
* `sigma_a`: `real<lower=0,upper=100>`, standard deviation of the offset for all locus tags
* `sigma_b`: `real<lower=0,upper=100>`, standard deviation of the slope for all locus tag
* `sigma_g`: `real<lower=0,upper=100>`, standard deviation of the offset for all locus tags due to *treatment*
* `sigma_d`: `real<lower=0,upper=100>`, standard deviation of the slope for all locus tags due to *treatment*


We also define a `transformed parameter`:

* `y_hat[i] <- b[tag[i]] * x[i] + a[tag[i]] + g[tag[i]] * t[i] + d[tag[i]] * t[i] * x[i]`: the linear relationship describing $\hat{y}$, our estimate of experimental output intensity, which is subject to variance `sigma`.

We define the model as:

$$\sigma_{\alpha} \sim U(0, 100)$$
$$\alpha_{j[i]} \sim Cauchy(\mu_{\alpha}, \sigma_{\alpha}^2)$$
$$\sigma_{\beta} \sim U(0, 100)$$
$$\beta_{j[i]} \sim Cauchy(\mu_{\beta}, \sigma_{\beta}^2)$$
$$\sigma_{\gamma} \sim U(0, 100)$$
$$\gamma_{j[i]} \sim Cauchy(\mu_{\gamma}, \sigma_{\gamma}^2)$$
$$\sigma_{\delta} \sim U(0, 100)$$
$$\delta_{j[i]} \sim Cauchy(\mu_{\delta}, \sigma_{\delta}^2)$$
$$y \sim N(\hat{y}, \sigma^2)$$


In [2]:
# load clean, normalised, indexed data
data = pd.read_csv("datasets/normalised_array_data.tab", sep="\t")  # full dataset

# useful values
locus_tags = data['locus_tag'].unique()
ntags = len(locus_tags)

In [3]:
# Create output directory and filename
outdir = "model_fits"
os.makedirs(outdir, exist_ok=True)
outfile = os.path.join(outdir, 'full_model_fit.pkl')

In [ ]:
# define unpooled stan model
treatment_model = """
data {
  int<lower=0> N;
  int<lower=0> J;
  int<lower=1, upper=J> tag[N];
  vector[N] t;
  vector[N] x;
  vector[N] y;
}
parameters {
  vector[J] a;
  vector[J] b;
  vector[J] g;
  vector[J] d;
  real mu_a;
  real mu_b;
  real mu_g;
  real mu_d;
  real<lower=0> sigma;
  real<lower=0,upper=100> sigma_a;
  real<lower=0,upper=100> sigma_b;
  real<lower=0,upper=100> sigma_g;
  real<lower=0,upper=100> sigma_d;
}
transformed parameters{
  vector[N] y_hat;

  for (i in 1:N)
    y_hat[i] = a[tag[i]] + b[tag[i]] * x[i] + g[tag[i]] * t[i] + d[tag[i]] * t[i] * x[i];
}
model {
  sigma_a ~ uniform(0, 100);
  a ~ cauchy(mu_a, sigma_a);

  sigma_b ~ uniform(0, 100);
  b ~ cauchy(mu_b, sigma_b);

  sigma_g ~ uniform(0, 100);
  g ~ cauchy(mu_g, sigma_g);

  sigma_d ~ uniform(0, 100);
  d ~ cauchy(mu_d, sigma_d);

  y ~ normal(y_hat, sigma);
}
"""

In [4]:
# relate python variables to stan variables
treatment_data_dict = {'N': len(data),
                       'J': ntags,
                       'tag': data['locus_tag_index'] + 1,
                       't': data['treatment'],
                       'x': data['log_input'],
                       'y': data['log_output']}

In [ ]:
# run stan fit
treatment_fit = pystan.stan(model_code=treatment_model,
                            data=treatment_data_dict,
                            iter=1000, chains=2,
                            seed=tools.SEED)

In [ ]:
# Save model to file, for reuse# Save fit to file, for reuse
unpermutedChains = treatment_fit.extract()
unpermutedChains_df = pd.DataFrame([dict(unpermutedChains)])
pickle.dump(unpermutedChains_df, open(outfile, 'wb'))

In [5]:
# Uncomment the line below to import a previously-fit model
treatment_fit = pickle.load(open(outfile, 'rb'))

In [6]:
# Get fits to alpha, beta, gamma by probe ID
alpha_estimates = pd.Series(treatment_fit['a'].mean(0), index=locus_tags)
alpha_se = pd.Series(treatment_fit['a'].std(0), index=locus_tags)
beta_estimates = pd.Series(treatment_fit['b'].mean(0), index=locus_tags)
beta_se = pd.Series(treatment_fit['b'].std(0), index=locus_tags)
gamma_estimates = pd.Series(treatment_fit['g'].mean(0), index=locus_tags)
gamma_se = pd.Series(treatment_fit['g'].std(0), index=locus_tags)
delta_estimates = pd.Series(treatment_fit['d'].mean(0), index=locus_tags)
delta_se = pd.Series(treatment_fit['d'].std(0), index=locus_tags)

Exception: Data must be 1-dimensional

In [ ]:
# Plot means distribution
g = sns.boxplot(delta_estimates)
g.set_title("Distribution of mean slope")
g.set_xlabel("mean(delta)");

In [ ]:
# Plot means distribution
g = sns.boxplot(delta_se)
g.set_title("Distribution of slope standard error")
g.set_xscale("log")
g.set_xlabel("se(delta)");

In [ ]:
# errors for slope outliers
tools.plot_threshold_errors(delta_estimates, delta_se, 0)

In [ ]:
delta_means = pd.DataFrame(delta_estimates)
delta_means.reset_index(inplace=True)
delta_means.columns = ['locus_tag', 'delta_mean']
delta_errors = pd.DataFrame(delta_se)
delta_errors.reset_index(inplace=True)
delta_errors.columns = ['locus_tag', 'delta_se']
out = pd.merge(delta_means, delta_errors, 'inner', ['locus_tag']).sort('locus_tag')
out.head()

In [ ]:
# plot predicted treatment effect
plt.figure(figsize=(20,8))
#plt.scatter(range(len(test_data)), test_data['beta'], alpha=1, color='k')
plt.scatter(range(len(out)), out['delta_mean'])
for idx, pmn, pse in zip(range(len(out)),
                              out['delta_mean'],
                              out['delta_se']):
    if abs(pmn) - abs(pse) > 0:
        color = 'b-'
        if pmn > 0:
            color = 'g-'
    else:
        color = 'r-'
    plt.plot([idx, idx], [pmn - pse, pmn + pse], color, alpha=0.4)
    #plt.plot([idx, out['beta']])
plt.xlim(2000, len(out))
#plt.xlim(4800, 5000)
#plt.ylim(0.4, 1.2)

In [ ]:
# Identify significant delta locus tags
out['significant'] = abs(out['delta_mean']) - abs(out['delta_se']) > 0
significant = out.loc[out['significant']]

In [ ]:
# Which locus tags have a significantly positive value of delta?
positives = significant.loc[significant['delta_mean'] > 0]
len(positives)

In [ ]:
list(positives['locus_tag'])